#1. Set Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


#2. Setup for analysis in R

In [2]:
%load_ext rpy2.ipython

In [3]:
%%R
pkgs_list = c("caret", "data.table",
               'stats', 'foreign',
               'nnet',  'readxl', 'pROC', 'base');
new_pkgs = pkgs_list[!(pkgs_list %in% installed.packages()[,"Package"])];
if(length(new_pkgs)) install.packages(new_pkgs);

R[write to console]: Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘future.apply’, ‘progressr’, ‘numDeriv’, ‘SQUAREM’, ‘lava’, ‘prodlim’, ‘proxy’, ‘iterators’, ‘gower’, ‘ipred’, ‘timeDate’, ‘e1071’, ‘foreach’, ‘ModelMetrics’, ‘plyr’, ‘recipes’, ‘reshape2’


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/listenv_0.8.0.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 36762 bytes (35 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to cons

In [4]:
%%R
library('pROC');
library('stats');
library('data.table');
library('foreign');
library('caret');
library('base')
library('nnet');
library("readxl");
#library('Metrics');

R[write to console]: Type 'citation("pROC")' for a citation.

R[write to console]: 
Attaching package: ‘pROC’


R[write to console]: The following objects are masked from ‘package:stats’:

    cov, smooth, var


R[write to console]: data.table 1.14.2 using 1 threads (see ?getDTthreads).  Latest news: r-datatable.com

R[write to console]: Loading required package: ggplot2

R[write to console]: Loading required package: lattice



#3. Load long data in R

In [5]:
%%R
# Set Long format folder as working directory to load dataframes : 
wd = setwd("/content/drive/Shareddrives/Eye_Tracking_NN_SD/Image_processing/Publishable_material/Datasets_regressions/Long_format")

In [6]:
%%R
# load data 
mydata2s = as.data.frame(read_excel("Dataset_regression_2s.xlsx"))
mydata5s = as.data.frame(read_excel("Dataset_regression_5s.xlsx"))
mydata10s = as.data.frame(read_excel("Dataset_regression_10s.xlsx"))
mydata15s= as.data.frame(read_excel("Dataset_regression_15s.xlsx"))
mydata15p=as.data.frame(read_excel("Dataset_regression_15p.xlsx"))
mydata30p=as.data.frame(read_excel("Dataset_regression_30p.xlsx"))
mydata50p=as.data.frame(read_excel("Dataset_regression_50p.xlsx"))
mydata80p=as.data.frame(read_excel("Dataset_regression_80p.xlsx"))
mydatafull=as.data.frame(read_excel("Dataset_regression_full.xlsx"))

In [7]:
%%R
# remove column 21 - R reads 'Intra-cell' like 'Intracell'. We remove 'Intra-cell' column.
mydata2s = mydata2s[,-21]
mydata5s = mydata5s[,-21]
mydata10s = mydata10s[,-21]
mydata15s = mydata15s[,-21]
mydata15p = mydata15p[,-21]
mydata30p = mydata30p[,-21]
mydata50p = mydata50p[,-21]
mydata80p = mydata80p[,-21]
mydatafull = mydatafull[,-21]

In [8]:
%%R
# Convert categorical variable to factor in each dataset:
mydata2s$Factors = as.numeric(as.factor(mydata2s$Levels))
mydata5s$Factors = as.numeric(as.factor(mydata5s$Levels))
mydata10s$Factors = as.numeric(as.factor(mydata10s$Levels))
mydata15s$Factors = as.numeric(as.factor(mydata15s$Levels))
mydata15p$Factors = as.numeric(as.factor(mydata15p$Levels))
mydata30p$Factors = as.numeric(as.factor(mydata30p$Levels))
mydata50p$Factors = as.numeric(as.factor(mydata50p$Levels))
mydata80p$Factors = as.numeric(as.factor(mydata80p$Levels))
mydatafull$Factors = as.numeric(as.factor(mydatafull$Levels))

#4. Fit and evaluate Multinomial-Logit model with each dataset 

In [9]:
%%R
setwd('/content/drive/Shareddrives/Eye_Tracking_NN_SD/Image_processing/Publishable_material/Models_performance/Logit/ROC_Strategic_sophistication')

In [10]:
%%R
#### 2 seconds ####
# Model fit:
fit_data <- mydata2s[which(mydata2s$data_collection=="first"),]
eval_data <- mydata2s[which(mydata2s$data_collection=="second"),]
# Full model:
mod <- multinom(Factors ~ as.numeric(Own) + as.numeric(Other) + as.numeric(Intracell.1), fit_data)
summary(mod)
# Predictive ability:
fitted.results = predict(mod, eval_data, type="class")
cm = confusionMatrix(as.factor(fitted.results), as.factor(eval_data$Factors))
overall = as.matrix(cm,what="overall")
overall_accuracy = overall[1,]
m = as.matrix(cm, what = "classes")
table = table(mydata2s$Factors)
n = nrow(mydata2s)
frequencies = c(table[1]/n, table[2]/n, table[3]/n)
m[is.na(m)] = 0
pr = m[5,]%*%frequencies  # WEIGHTED AVERAGE
f1 = m[7,]%*%frequencies  # WEIGHTED AVERAGE
roc.multi = multiclass.roc(eval_data$Factors, as.numeric(fitted.results), levels = c(1,2,3))
average_auc = auc(roc.multi)
outcome_2s = cbind(overall_accuracy, average_auc, pr, f1)
colnames(outcome_2s) = c("Accuracy", "AUC", "Precision", "f1")
print(outcome_2s)
rs <- roc.multi[['rocs']]
png(file="roc_2s.png",width=600, height=350)
plot.roc(rs[[1]])
sapply(2:length(rs),function(i) lines.roc(rs[[i]],col=i))
dev.off()
rm(fit_data, eval_data, mod, fitted.results)

# weights:  15 (8 variable)
initial  value 1549.043327 
iter  10 value 1297.687262
final  value 1297.551681 
converged


R[write to console]: Setting direction: controls < cases

R[write to console]: Setting direction: controls < cases

R[write to console]: Setting direction: controls < cases



         Accuracy       AUC Precision        f1
Accuracy 0.545098 0.5610551 0.4207736 0.4775094


In [11]:
%%R
#### 5 seconds ####
# Model fit:
fit_data <- mydata5s[which(mydata5s$data_collection=="first"),]
eval_data <- mydata5s[which(mydata5s$data_collection=="second"),]
# Full model:
mod <- multinom(Factors ~ as.numeric(Own) + as.numeric(Other) + as.numeric(Intracell.1), fit_data)
summary(mod)
# Predictive ability:
fitted.results = predict(mod, eval_data, type="class")
cm = confusionMatrix(as.factor(fitted.results), as.factor(eval_data$Factors))
overall = as.matrix(cm,what="overall")
overall_accuracy = overall[1,]
m = as.matrix(cm, what = "classes")
table = table(mydata5s$Factors)
n = nrow(mydata5s)
frequencies = c(table[1]/n, table[2]/n, table[3]/n)
m[is.na(m)] = 0
pr = m[5,]%*%frequencies  # WEIGHTED AVERAGE
f1 = m[7,]%*%frequencies  # WEIGHTED AVERAGE
roc.multi = multiclass.roc(eval_data$Factors, as.numeric(fitted.results), levels = c(1,2,3))
average_auc = auc(roc.multi)
outcome_5s = cbind(overall_accuracy, average_auc, pr, f1)
colnames(outcome_5s) = c("Accuracy", "AUC", "Precision", "f1")
print(outcome_5s)
rs <- roc.multi[['rocs']]
png(file="roc_5s.png",width=600, height=350)
plot.roc(rs[[1]])
sapply(2:length(rs),function(i) lines.roc(rs[[i]],col=i))
dev.off()
rm(fit_data,eval_data, mod, fitted.results)

# weights:  15 (8 variable)
initial  value 1549.043327 
iter  10 value 1245.284196
final  value 1245.261121 
converged


R[write to console]: Setting direction: controls > cases

R[write to console]: Setting direction: controls < cases

R[write to console]: Setting direction: controls < cases



          Accuracy       AUC Precision       f1
Accuracy 0.5627451 0.6263532 0.4380678 0.496586


In [12]:
%%R
#### 10 seconds ####
# Model fit:
fit_data <- mydata10s[which(mydata10s$data_collection=="first"),]
eval_data <- mydata10s[which(mydata10s$data_collection=="second"),]
# Full model:
mod <- multinom(Factors ~ as.numeric(Own) + as.numeric(Other) + as.numeric(Intracell.1), fit_data)
summary(mod)
# Predictive ability:
fitted.results = predict(mod, eval_data, type="class")
cm = confusionMatrix(as.factor(fitted.results), as.factor(eval_data$Factors))
overall = as.matrix(cm,what="overall")
overall_accuracy = overall[1,]
m = as.matrix(cm, what = "classes")
table = table(mydata10s$Factors)
n = nrow(mydata10s)
frequencies = c(table[1]/n, table[2]/n, table[3]/n)
m[is.na(m)] = 0
pr = m[5,]%*%frequencies  # WEIGHTED AVERAGE
f1 = m[7,]%*%frequencies  # WEIGHTED AVERAGE
roc.multi = multiclass.roc(eval_data$Factors, as.numeric(fitted.results), levels = c(1,2,3))
average_auc = auc(roc.multi)
outcome_10s = cbind(overall_accuracy, average_auc, pr, f1)
colnames(outcome_10s) = c("Accuracy", "AUC", "Precision", "f1")
print(outcome_10s)
rs <- roc.multi[['rocs']]
png(file="roc_10s.png",width=600, height=350)
plot.roc(rs[[1]])
sapply(2:length(rs),function(i) lines.roc(rs[[i]],col=i))
dev.off()
rm(fit_data, eval_data, mod, fitted.results)

# weights:  15 (8 variable)
initial  value 1549.043327 
iter  10 value 1230.438298
final  value 1230.358778 
converged


R[write to console]: Setting direction: controls > cases

R[write to console]: Setting direction: controls < cases

R[write to console]: Setting direction: controls < cases



          Accuracy       AUC Precision        f1
Accuracy 0.5813725 0.6345635 0.4523421 0.5099523


In [13]:
%%R
#### 15 seconds ####
# Model fit:
fit_data <- mydata15s[which(mydata15s$data_collection=="first"),]
eval_data <- mydata15s[which(mydata15s$data_collection=="second"),]
# Full model:
mod <- multinom(Factors ~ as.numeric(Own) + as.numeric(Other) + as.numeric(Intracell.1), fit_data)
# Predictive ability:
fitted.results = predict(mod, eval_data, type="class")
cm = confusionMatrix(as.factor(fitted.results), as.factor(eval_data$Factors))
overall = as.matrix(cm,what="overall")
overall_accuracy = overall[1,]
m = as.matrix(cm, what = "classes")
table = table(mydata15s$Factors)
n = nrow(mydata15s)
frequencies = c(table[1]/n, table[2]/n, table[3]/n)
m[is.na(m)] = 0
pr = m[5,]%*%frequencies  # WEIGHTED AVERAGE
f1 = m[7,]%*%frequencies  # WEIGHTED AVERAGE
roc.multi = multiclass.roc(eval_data$Factors, as.numeric(fitted.results), levels = c(1,2,3))
average_auc = auc(roc.multi)
outcome_15s = cbind(overall_accuracy, average_auc, pr, f1)
colnames(outcome_15s) = c("Accuracy", "AUC", "Precision", "f1")
print(outcome_15s)
rs <- roc.multi[['rocs']]
png(file="roc_15s.png",width=600, height=350)
plot.roc(rs[[1]])
sapply(2:length(rs),function(i) lines.roc(rs[[i]],col=i))
dev.off()
rm(fit_data, eval_data, mod, fitted.results)

# weights:  15 (8 variable)
initial  value 1549.043327 
iter  10 value 1218.930540
final  value 1218.771805 
converged


R[write to console]: Setting direction: controls > cases

R[write to console]: Setting direction: controls < cases

R[write to console]: Setting direction: controls < cases



          Accuracy       AUC Precision        f1
Accuracy 0.5911765 0.6391174  0.458899 0.5164693


In [14]:
%%R
#### Full images ####
# Model fit:
fit_data <- mydatafull[which(mydatafull$data_collection=="first"),]
eval_data <- mydatafull[which(mydatafull$data_collection=="second"),]
# Full model:
mod <- multinom(Factors ~ as.numeric(Own) + as.numeric(Other) + as.numeric(Intracell), fit_data)
summary(mod)
# Predictive ability:
fitted.results = predict(mod, eval_data, type="class")
cm = confusionMatrix(as.factor(fitted.results), as.factor(eval_data$Factors))
overall = as.matrix(cm,what="overall")
overall_accuracy = overall[1,]
m = as.matrix(cm, what = "classes")
table = table(mydatafull$Factors)
n = nrow(mydatafull)
frequencies = c(table[1]/n, table[2]/n, table[3]/n)
m[is.na(m)] = 0
pr = m[5,]%*%frequencies  # WEIGHTED AVERAGE
f1 = m[7,]%*%frequencies  # WEIGHTED AVERAGE
roc.multi = multiclass.roc(eval_data$Factors, as.numeric(fitted.results), levels = c(1,2,3))
average_auc = auc(roc.multi)
outcome_full = cbind(overall_accuracy, average_auc, pr, f1)
colnames(outcome_full) = c("Accuracy", "AUC", "Precision", "f1")
print(outcome_full)
rs <- roc.multi[['rocs']]
png(file="roc_full.png",width=600, height=350)
plot.roc(rs[[1]])
sapply(2:length(rs),function(i) lines.roc(rs[[i]],col=i))
dev.off()
rm(fit_data, eval_data, mod, fitted.results)

# weights:  15 (8 variable)
initial  value 1549.043327 
iter  10 value 1200.900203
final  value 1200.642629 
converged


R[write to console]: Setting direction: controls < cases

R[write to console]: Setting direction: controls < cases

R[write to console]: Setting direction: controls < cases



          Accuracy       AUC Precision        f1
Accuracy 0.5970588 0.5786653 0.4650194 0.5207781


In [15]:
%%R
#### 80 percent ####
# Model fit:
fit_data <- mydata80p[which(mydata80p$data_collection=="first"),]
eval_data <- mydata80p[which(mydata80p$data_collection=="second"),]
# Full model:
mod <- multinom(Factors ~ as.numeric(Own) + as.numeric(Other) + as.numeric(Intracell.1), fit_data)
summary(mod)
# Predictive ability:
fitted.results = predict(mod, eval_data, type="class")
cm = confusionMatrix(as.factor(fitted.results), as.factor(eval_data$Factors))
overall = as.matrix(cm,what="overall")
overall_accuracy = overall[1,]
m = as.matrix(cm, what = "classes")
table = table(mydata80p$Factors)
n = nrow(mydata80p)
frequencies = c(table[1]/n, table[2]/n, table[3]/n)
m[is.na(m)] = 0
pr = m[5,]%*%frequencies  # WEIGHTED AVERAGE
f1 = m[7,]%*%frequencies  # WEIGHTED AVERAGE
roc.multi = multiclass.roc(eval_data$Factors, as.numeric(fitted.results), levels = c(1,2,3))
average_auc = auc(roc.multi)
outcome_80p = cbind(overall_accuracy, average_auc, pr, f1)
colnames(outcome_80p) = c("Accuracy", "AUC", "Precision", "f1")
print(outcome_80p)
rs <- roc.multi[['rocs']]
png(file="roc_80p.png",width=600, height=350)
plot.roc(rs[[1]])
sapply(2:length(rs),function(i) lines.roc(rs[[i]],col=i))
dev.off()
rm(fit_data, eval_data, mod, fitted.results)

# weights:  15 (8 variable)
initial  value 1549.043327 
iter  10 value 1205.240295
final  value 1205.150445 
converged


R[write to console]: Setting direction: controls > cases

R[write to console]: Setting direction: controls < cases

R[write to console]: Setting direction: controls < cases



         Accuracy       AUC Precision        f1
Accuracy      0.6 0.6467097 0.4671003 0.5243478


In [16]:
%%R
#### 50 percent ####
# Model fit:
fit_data <- mydata50p[which(mydata50p$data_collection=="first"),]
eval_data <- mydata50p[which(mydata50p$data_collection=="second"),]
# Full model:
mod <- multinom(Factors ~ as.numeric(Own) + as.numeric(Other) + as.numeric(Intracell.1), fit_data)
summary(mod)
# Predictive ability:
fitted.results = predict(mod, eval_data, type="class")
cm = confusionMatrix(as.factor(fitted.results), as.factor(eval_data$Factors))
overall = as.matrix(cm,what="overall")
overall_accuracy = overall[1,]
m = as.matrix(cm, what = "classes")
table = table(mydata50p$Factors)
n = nrow(mydata50p)
frequencies = c(table[1]/n, table[2]/n, table[3]/n)
m[is.na(m)] = 0
pr = m[5,]%*%frequencies  # WEIGHTED AVERAGE
f1 = m[7,]%*%frequencies  # WEIGHTED AVERAGE
roc.multi = multiclass.roc(eval_data$Factors, as.numeric(fitted.results), levels = c(1,2,3))
average_auc = auc(roc.multi)
outcome_50p = cbind(overall_accuracy, average_auc, pr, f1)
colnames(outcome_50p) = c("Accuracy", "AUC", "Precision", "f1")
print(outcome_50p)
rs <- roc.multi[['rocs']]
png(file="roc_50p.png",width=600, height=350)
plot.roc(rs[[1]])
sapply(2:length(rs),function(i) lines.roc(rs[[i]],col=i))
dev.off()
rm(fit_data, eval_data, mod, fitted.results, roc.multi)

# weights:  15 (8 variable)
initial  value 1549.043327 
iter  10 value 1231.467931
final  value 1231.447111 
converged


R[write to console]: Setting direction: controls > cases

R[write to console]: Setting direction: controls < cases

R[write to console]: Setting direction: controls < cases



          Accuracy       AUC Precision        f1
Accuracy 0.5813725 0.6359653 0.4510662 0.5105236


In [17]:
%%R
#### 30 percent ####
# Model fit:
fit_data <- mydata30p[which(mydata30p$data_collection=="first"),]
eval_data <- mydata30p[which(mydata30p$data_collection=="second"),]
# Full model:
mod <- multinom(Factors ~ as.numeric(Own) + as.numeric(Other) + as.numeric(Intracell.1), data= fit_data)
summary(mod)
# Predictive ability:
fitted.results = predict(mod, eval_data, type="class")
cm = confusionMatrix(as.factor(fitted.results), as.factor(eval_data$Factors))
overall = as.matrix(cm,what="overall")
overall_accuracy = overall[1,]
m = as.matrix(cm, what = "classes")
table = table(mydata30p$Factors)
n = nrow(mydata30p)
frequencies = c(table[1]/n, table[2]/n, table[3]/n)
m[is.na(m)] = 0
pr = m[5,]%*%frequencies  # WEIGHTED AVERAGE
f1 = m[7,]%*%frequencies  # WEIGHTED AVERAGE
roc.multi = multiclass.roc(eval_data$Factors, as.numeric(fitted.results), levels = c(1,2,3))
average_auc = auc(roc.multi)
outcome_30p = cbind(overall_accuracy, average_auc, pr, f1)
colnames(outcome_30p) = c("Accuracy", "AUC", "Precision", "f1")
print(outcome_30p)
rs <- roc.multi[['rocs']]
png(file="roc_30p.png",width=600, height=350)
plot.roc(rs[[1]])
sapply(2:length(rs),function(i) lines.roc(rs[[i]],col=i))
dev.off()
rm(fit_data, eval_data, mod, fitted.results)

# weights:  15 (8 variable)
initial  value 1544.648878 
iter  10 value 1278.108358
final  value 1278.094698 
converged


R[write to console]: Setting direction: controls > cases

R[write to console]: Setting direction: controls < cases

R[write to console]: Setting direction: controls < cases



          Accuracy      AUC Precision        f1
Accuracy 0.5578431 0.618505 0.5399716 0.4926722


In [18]:
%%R
#### 15 percent ####
# Model fit:
fit_data <- mydata15p[which(mydata15p$data_collection=="first"),]
eval_data <- mydata15p[which(mydata15p$data_collection=="second"),]
# Full model:
mod <- multinom(Factors ~ as.numeric(Own) + as.numeric(Other) + as.numeric(Intracell.1), fit_data)
summary(mod)
# Predictive ability:
fitted.results = predict(mod, eval_data, type="class")
cm = confusionMatrix(as.factor(fitted.results), as.factor(eval_data$Factors))
overall = as.matrix(cm,what="overall")
overall_accuracy = overall[1,]
m = as.matrix(cm, what = "classes")
table = table(mydata15p$Factors)
n = nrow(mydata15p)
frequencies = c(table[1]/n, table[2]/n, table[3]/n)
m[is.na(m)] = 0
pr = m[5,]%*%frequencies  # WEIGHTED AVERAGE
f1 = m[7,]%*%frequencies  # WEIGHTED AVERAGE
roc.multi = multiclass.roc(eval_data$Factors, as.numeric(fitted.results), levels = c(1,2,3))
average_auc = auc(roc.multi)
outcome_15p = cbind(overall_accuracy, average_auc, pr, f1)
colnames(outcome_15p) = c("Accuracy", "AUC", "Precision", "f1")
print(outcome_15p)
rs <- roc.multi[['rocs']]
png(file="roc_15p.png",width=600, height=350)
plot.roc(rs[[1]])
sapply(2:length(rs),function(i) lines.roc(rs[[i]],col=i))
dev.off()
rm(fit_data, eval_data, mod, fitted.results)

# weights:  15 (8 variable)
initial  value 1528.169694 
iter  10 value 1305.156560
final  value 1304.281051 
converged


R[write to console]: Setting direction: controls < cases

R[write to console]: Setting direction: controls < cases

R[write to console]: Setting direction: controls < cases



          Accuracy       AUC Precision        f1
Accuracy 0.5376238 0.5464642 0.4173123 0.4737933


#5. Save performance results

In [19]:
%%R
#### save results ####
setwd('/content/drive/Shareddrives/Eye_Tracking_NN_SD/Image_processing/Publishable_material/Models_performance/Logit')

outcome_timing = rbind(outcome_full, outcome_15s, outcome_10s, outcome_5s, outcome_2s)
Dataset1 = c ('full', '15s', '10s', '5s', '2s')
outcome_timing = as.data.frame(cbind(Dataset1, outcome_timing))
write.csv(outcome_timing, file = "outcome_Multinom_timing.csv")


outcome_percentage = rbind(outcome_full, outcome_80p, outcome_50p, outcome_30p, outcome_15p)
Dataset2 = c ('full', '80', '50', '30', '15' )
outcome_percentage = as.data.frame(cbind(Dataset2, outcome_percentage))
write.csv(outcome_percentage , file = "outcome_Multinom_percentage.csv")

In [20]:
%%R
outcome_timing

           Dataset1          Accuracy               AUC         Precision
Accuracy       full 0.597058823529412 0.578665257851667 0.465019394934398
Accuracy.1      15s 0.591176470588235 0.639117376838398 0.458899029381304
Accuracy.2      10s 0.581372549019608 0.634563527693078 0.452342064987733
Accuracy.3       5s 0.562745098039216 0.626353236372883 0.438067829877858
Accuracy.4       2s 0.545098039215686 0.561055070358202 0.420773647559723
                          f1
Accuracy   0.520778136094855
Accuracy.1 0.516469320545309
Accuracy.2 0.509952324042527
Accuracy.3 0.496585957692993
Accuracy.4 0.477509368111332


In [21]:
%%R
outcome_percentage

           Dataset2          Accuracy               AUC         Precision
Accuracy       full 0.597058823529412 0.578665257851667 0.465019394934398
Accuracy.1       80               0.6 0.646709653585882  0.46710033569937
Accuracy.2       50 0.581372549019608 0.635965322605794 0.451066231127495
Accuracy.3       30 0.557843137254902 0.618505046469749 0.539971613688103
Accuracy.4       15 0.537623762376238 0.546464152935494 0.417312342377645
                          f1
Accuracy   0.520778136094855
Accuracy.1 0.524347849418141
Accuracy.2 0.510523566499139
Accuracy.3 0.492672180929087
Accuracy.4 0.473793290878975
